<a href="https://colab.research.google.com/github/jvrscak/UUIProjekt/blob/main/drugi_zadatak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DRUGI ZADATAK - simulacija prometa

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import random

N = 10          # duljina cesta
M = 12          # broj stanica kružnog toka
TIME_STEPS = 200
MAX_SPEED = 2
ULAZNA_VJEROVATNOST = 0.25  # vjerojatnost pojavljivanja novog vozila po koraku
MIN_SPAWN_INTERVAL = 2      # minimalni interval između pojavljivanja vozila

SEMAFOR_CYCLE = 20          # ciklus semafora (koraci)
SEMAFOR_GREEN_TIME = 12     # vrijeme zelenog svjetla (koraci)

ROUNDABOUT_RADIUS = 3
theta = np.linspace(0, 2*np.pi, M, endpoint=False)
round_cells = [(ROUNDABOUT_RADIUS*np.cos(a), ROUNDABOUT_RADIUS*np.sin(a)) for a in theta]

# Ceste (ulazi i izlazi)
cestaA = {i: None for i in range(N)}
cestaB = {i: None for i in range(N)}
cestaC = {i: None for i in range(N)}
cestaD = {i: None for i in range(N)}

# Ceste u suprotnom smjeru (izlazi iz kružnog toka)
cestaA_reverse = {i: None for i in range(N)}
cestaB_reverse = {i: None for i in range(N)}
cestaC_reverse = {i: None for i in range(N)}
cestaD_reverse = {i: None for i in range(N)}

# Automobili
road_cars = {}
round_cars = {}

# Semafori
semafori = {
    'A': (1, 0),  # (faza, timer)
    'B': (0, 0),
    'C': (0, 0),
    'D': (0, 0),
    'A_reverse': (1, 0),
    'B_reverse': (0, 0),
    'C_reverse': (0, 0),
    'D_reverse': (0, 0)
}

# Brojač za ID vozila
next_car_id = 0

# Zadnje vrijeme pojavljivanja vozila po ulazu
last_spawn_time = {'A': -MIN_SPAWN_INTERVAL, 'B': -MIN_SPAWN_INTERVAL,
                   'C': -MIN_SPAWN_INTERVAL, 'D': -MIN_SPAWN_INTERVAL,
                   'A_reverse': -MIN_SPAWN_INTERVAL, 'B_reverse': -MIN_SPAWN_INTERVAL,
                   'C_reverse': -MIN_SPAWN_INTERVAL, 'D_reverse': -MIN_SPAWN_INTERVAL}

def next_round_index(idx, speed):
    """Pomak u kružnom toku"""
    return (idx + speed) % M

def distance_next_round(round_cars, idx):
    """Udaljenost do sljedećeg vozila u kružnom toku"""
    positions = [pos for pos, _, _, _ in round_cars.values()]
    gap = 0
    cur = idx
    while True:
        cur = (cur + 1) % M
        gap += 1
        if cur in positions or gap > M:
            break
    return gap - 1

def distance_next_road(road_cars, pos, cesta_name):
    """Udaljenost do sljedećeg vozila na cesti"""
    # Za normalne ceste (A,B,C,D): vozila idu od N-1 (van) prema 0 (kružni tok) - ULAZ
    # Za reverse ceste (A_reverse,B_reverse,C_reverse,D_reverse): vozila idu od 0 (kružni tok) prema N-1 (van) - IZLAZ
    is_reverse = '_reverse' in cesta_name

    if is_reverse:
        # Na reverse cesti, vozila idu naprijed (od 0 prema N-1) - izlaz iz kružnog toka
        occupied = [p for c, p, _, _, _ in road_cars.values() if c == cesta_name and p > pos]
        if occupied:
            return min(occupied) - pos - 1
        else:
            return N - pos - 1  # Do kraja ceste (pozicija N-1)
    else:
        # Na normalnoj cesti, vozila idu unazad (od N-1 prema 0) - ulaz u kružni tok
        occupied = [p for c, p, _, _, _ in road_cars.values() if c == cesta_name and p < pos]
        if occupied:
            return pos - max(occupied) - 1
        else:
            return pos  # Do kraja ceste (pozicija 0 - ulaz u kružni tok)